In [1]:
%matplotlib notebook

import matplotlib.pyplot as plt
import os as os
import sys as sys
import glob as glob
import re as re
import numpy as np
import shutil as shutil

import paprika
print(paprika.__version__)

from paprika.amber import Simulation
from paprika import analysis
from paprika.restraints import DAT_restraint

import pytraj as pt
import parmed as pmd

2018-04-23_16:10:29_-0700-e34840d-0.0.3


In [2]:
import logging
from importlib import reload
reload(logging)

logging.basicConfig(filename='OA-G3.log', format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %I:%M:%S %p',
                    level=logging.DEBUG)
logging.info('Started logging...')
logging.info(paprika.__version__)

In [3]:
dummy_anchors = [':3', ':4', ':5']
host_anchors  = [':1@C26', ':1@C18', ':1@C8']
guest_anchors = [':2@C1', ':2@C3']

In [4]:
attach_string = '0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 ' \
                '18.10 24.40 37.00 49.60 74.80 100.00'
attach_fractions = [float(i) / 100 for i in attach_string.split()]

pull_string = '0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 ' \
              '4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 ' \
              '8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 ' \
              '12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 ' \
              '16.80 17.20 17.60 18.00'

windows = [len(attach_fractions), len(pull_string.split()), 0]


In [5]:
g3_complexes = glob.glob('./OA-G3*')
ntwprt = 204
itr_nstlim = 500000

paths = glob.glob(os.path.normpath(os.path.join('./OA-G3-0', 'AMBER', 'APR', 'windows', '*')))

window_list = [os.path.basename(i) for i in paths]
phase_dict = {'a': 'attach', 'p': 'pull', 'r': 'release'}

# End preamble

In [10]:
fig,axs = plt.subplots(figsize=(9,4))
axis = [4.0, 26.0, 0.0, 2.5]
zorder = len(window_list) + 2

# Converence threshold, kcal/mol (probably need separate for attach vs. pull)
convergence_thresh = {'attach': 0.006, 'pull': 0.03}
# Simulation steps per iteration


<IPython.core.display.Javascript object>

In [117]:
for complx in g3_complexes:
    
    traj = pt.load(os.path.join(complx, 'AMBER', 'solvate.rst7'), 
                    os.path.join(complx, 'AMBER', 'solvate.prmtop'))

    pull_initial = pt.distance(traj, ' '.join([dummy_anchors[0], guest_anchors[0]]))[0]

    print(f'Set pull offset ({pull_initial:1.2f} A)')

    pull_distances = [float(i) + pull_initial for i in pull_string.split()]
    
    guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                        ]

    guest_restraint_targets = [pull_initial,
                               180.0,
                               180.0
                              ]
    guest_restraint_target_final = [pull_distances[-1], 180.0, 180.0]
    guest_restraint_distance_fc = 5.0 # kcal/mol-A**2
    guest_restraint_angle_fc = 100.0  # kcal/mol-rad**2

    hg = pmd.load_file(os.path.join(complx, 'AMBER', 'solvate.prmtop'),
                   os.path.join(complx, 'AMBER', 'solvate.rst7'),
                   structure=True)

    guest_restraints = []
    for index, atoms in enumerate(guest_restraint_atoms): 
        if len(atoms) > 2:
            angle = True
        else:
            angle = False
        this = DAT_restraint()
        this.auto_apr = True
        this.amber_index = True
        this.topology = hg
        this.mask1 = atoms[0]
        this.mask2 = atoms[1]
        if angle:
            this.mask3 = atoms[2]
            this.attach['fc_final'] = guest_restraint_angle_fc
        else:
            this.attach['fc_final'] = guest_restraint_distance_fc
        this.attach['target'] = guest_restraint_targets[index]
        this.attach['fraction_list'] = attach_fractions

        this.pull['target_final'] = guest_restraint_target_final[index]
        this.pull['num_windows'] = windows[1]
        this.initialize()

        guest_restraints.append(this)
        
    structure = pt.load(os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'solvate.rst7'), 
                       os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'solvate.prmtop'))

    stripped = structure.strip(':WAT,:Na+,:Cl-')

    
    print(f'Simulating {complx}')

    # Store simulations here
    sim_list = []
    # Convergence check
    all_converged = {'attach': None, 'pull': None, 'release': None}

    # Loop of restart iterations
    for itr in range(50):
        # Checker for whether new data has been generated
        # Start with assumption of no new data
        new_data = False

        # Strings for restart iterations
        prev = "{:03.0f}".format(itr)
        curr = "{:03.0f}".format(itr+1)

        print('----- prev :'+prev+' cur: '+curr+' -----')
        sys.stdout.flush()

        # Loop through windows
        for i_win, win in enumerate(window_list):
            # Create some useful variables
            phase = phase_dict[win[0]]
            win_path = os.path.join(complx, 'AMBER', 'APR', 'windows', win)

            # If first time through, create simulation class and minimize
            if itr == 0:
                sim = Simulation()
                sim_list.append(sim)
                sim.executable = 'pmemd.cuda'
                sim.CUDA_VISIBLE_DEVICES = 1
                sim.path = win_path
                sim.topology = 'solvate.prmtop'
                sim.prefix = 'minimize'
                sim.inpcrd = 'solvate.rst7'
                sim.ref = 'solvate.rst7'
                sim.config_pbc_min()
                sim.cntrl['maxcyc'] = 500
                sim.cntrl['ncyc'] = 400
                sim.cntrl['ntr'] = 1
                sim.cntrl['restraint_wt'] = 50.0
                sim.cntrl['restraintmask'] = "'@DUM'"
                sim.restraint_file = 'disang.rest'
                # This might be an overwrite run, so check if minimize is done already
                if not os.path.isfile(win_path+'/minimize.rst7'):
                    print('Re-running minimization...')
                    sim.run(overwrite=True)

                # Run Equilibration ... if we terminate early due to
                # box size changes, continue on from restart
                sim.config_pbc_md()
                sim.executable = 'pmemd.cuda'
                sim.cntrl['nstlim'] = itr_nstlim
                sim.cntrl['ntwx'] = 250
                sim.cntrl['ntwprt'] = ntwprt
                sim.cntrl['ntwr'] = 250
                sim.restraint_file = 'disang.rest'
                i = 0
                sim.prefix = 'equil.'+str(i)
                sim.inpcrd = 'minimize.rst7'
                sim.ref = 'solvate.rst7'
                # Run equil loop until we complete a full equil cycle.
                # Then copy the final rst7 to 'equil.rst7'
                while not os.path.isfile(win_path+'/equil.rst7') and i < 10:
                    print('`equil.rst7` not found and i < 10...')
                    if not os.path.isfile(win_path+'/'+sim.inpcrd):
                        print('`sim.inpcrd` not found...')
                        break
                    sim.run(overwrite=True)
                    with open(win_path+'/equil.'+str(i)+'.out') as f:
                        for line in f.readlines():
                            if re.search(' TIMINGS', line):
                                shutil.copy(win_path+'/'+sim.restart, win_path+'/equil.rst7')
                    i += 1
                    sim.prefix = 'equil.'+str(i)
                    sim.inpcrd = 'equil.'+str(i-1)+'.rst7'
                    sim.ref = 'solvated.rst7'

            # Setup production MD
            if itr == 0:
                sim.config_pbc_md()
                sim.executable = 'pmemd.cuda'
                sim.prefix = 'prod.'+curr
                #sim.input = 'mdin'
                sim.inpcrd = 'equil.rst7'
                sim.ref = 'solvate.rst7'
                sim.cntrl['ntx'] = 5
                sim.cntrl['irest'] = 1
                sim.cntrl['nstlim'] = itr_nstlim
                sim.cntrl['ntwr'] = itr_nstlim
                sim.cntrl['ntwx'] = 250
                sim.cntrl['ntwprt'] = ntwprt
                sim.cntrl['ntxo'] = 2
                sim.restraint_file = 'disang.rest'
                if not os.path.isfile(win_path+'/'+sim.restart):
                    sim.run(overwrite=True)
                    new_data = True
                # From now on, don't assume we can access sim. Use
                # sim_list[i_win] instead. The reason is we aren't
                # gonna keep setting sim up each time and we've stored
                # them in sim_list
                sim = None            
            else:
                sim_list[i_win].prefix = 'prod.'+curr
                sim_list[i_win].inpcrd = 'prod.'+prev+'.rst7'
                sim_list[i_win].ref = 'solvate.rst7'
                #sim_list[i_win].input = 'mdin'
                # Check convergence to decide if we are "converged"
                convergence_val = fecalc.results[phase]['mbar-block']['convergence'][int(win[1:])]
                if convergence_val < convergence_thresh[phase]:
                    sim_list[i_win].converged = True
                if not sim_list[i_win].converged and not os.path.isfile(win_path+'/'+sim_list[i_win].restart):
                    sys.stdout.write(win+' ')
                    sim_list[i_win].run(overwrite=True)
                    new_data = True
        print('')            
        
        # Collect data and compute free energy                   
        if itr == 0 or new_data:
            fecalc = analysis.fe_calc()
            fecalc.prmtop = stripped.topology
            fecalc.trajectory = 'prod.*.nc'
            fecalc.path = os.path.join(complx, 'AMBER', 'APR', 'windows')
            fecalc.restraint_list = guest_restraints
            fecalc.collect_data(single_prmtop=True)
            fecalc.compute_free_energy()
            print('{}: Attach: {:.2f} ({:.2f})  Pull: {:.2f} ({:.2f})'.format(
                complx,
                fecalc.results['attach']['mbar-block']['fe'],
                fecalc.results['attach']['mbar-block']['sem'],
                fecalc.results['pull']['mbar-block']['fe'],
                fecalc.results['pull']['mbar-block']['sem'],
            ))

        # Check overall Convergence
        for phase in "attach pull".split():
            convergence_vals = np.array(fecalc.results[phase]['mbar-block']['convergence'])
            if (convergence_vals < 0.05).all():
                all_converged[phase] = True
            else:
                all_converged[phase] = False
        
        # Update our plot.
        axs.clear()
        axs.axis(axis)
        axs.set_title('{}: Attach: {:.2f} ({:.2f})  Pull: {:.2f} ({:.2f})'.format(
            conf_path,
            fecalc.results['attach']['mbar-block']['fe'],
            fecalc.results['attach']['mbar-block']['sem'],
            fecalc.results['pull']['mbar-block']['fe'],
            fecalc.results['pull']['mbar-block']['sem'],
        ))
        for phase in 'attach pull'.split():                    
            for i in range(len(fecalc.simulation_data[phase])):
                hist,bins = np.histogram(
                                fecalc.simulation_data[phase][i][0],
                                bins=301,
                                range=(0,30),
                                density=True
                            )
                center = (bins[:-1] + bins[1:]) / 2
                if phase == 'attach':
                    facecolor = 'r'
                else:
                    facecolor = 'c'
                axs.fill_between(
                    center, hist*0.0, hist, facecolor=facecolor, edgecolor='k',
                    alpha=0.3, zorder=zorder
                )
                zorder -= 1
            fig.canvas.draw()
            
        # If all phases are converged, we're done!
        if not False in all_converged.values():
            print("Done!")
            break

        sys.stdout.flush()

    

Set pull offset (8.03 A)


2018-04-24 05:24:08 PM Calculating attach targets and force constants...
2018-04-24 05:24:08 PM Attach, Method #3
2018-04-24 05:24:08 PM Calculating pull targets and force constants...
2018-04-24 05:24:08 PM Pull, Method #1
2018-04-24 05:24:08 PM Calculating release targets and force constants...
2018-04-24 05:24:08 PM No restraint info set for the release phase! Skipping...
2018-04-24 05:24:08 PM Number of attach windows = 15
2018-04-24 05:24:08 PM Number of pull windows = 46
2018-04-24 05:24:08 PM This restraint will be skipped in the release phase
2018-04-24 05:24:08 PM Assigning atom indices...
2018-04-24 05:24:08 PM There are 1 atoms in the mask :3  ...
2018-04-24 05:24:08 PM There are 1 atoms in the mask :2@C1  ...
2018-04-24 05:24:08 PM Calculating attach targets and force constants...
2018-04-24 05:24:08 PM Attach, Method #3
2018-04-24 05:24:08 PM Calculating pull targets and force constants...
2018-04-24 05:24:08 PM Pull, Method #1
2018-04-24 05:24:08 PM Calculating release ta

Simulating ./OA-G3-0
----- prev :000 cur: 001 -----


2018-04-24 05:24:08 PM Writing minimize.in
2018-04-24 05:24:08 PM Running Minimization at ./OA-G3-0/AMBER/APR/windows/a000
2018-04-24 05:24:08 PM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c solvate.rst7 -i minimize.in -o minimize.out -r minimize.rst7 -inf minimize.mdinfo


Re-running minimization...


...

2018-04-24 11:51:04 PM Minimization completed...
2018-04-24 11:51:04 PM Writing equil.0.in
2018-04-24 11:51:04 PM Running MD at ./OA-G3-0/AMBER/APR/windows/p045
2018-04-24 11:51:04 PM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c minimize.rst7 -i equil.0.in -o equil.0.out -r equil.0.rst7 -x equil.0.nc -inf equil.0.mdinfo -e equil.0.mden


`equil.rst7` not found and i < 10...


2018-04-24 11:54:55 PM MD completed ...
2018-04-24 11:54:55 PM Writing prod.001.in
2018-04-24 11:54:55 PM Running MD at ./OA-G3-0/AMBER/APR/windows/p045
2018-04-24 11:54:55 PM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c equil.rst7 -i prod.001.in -o prod.001.out -r prod.001.rst7 -x prod.001.nc -inf prod.001.mdinfo -e prod.001.mden
2018-04-24 11:57:39 PM MD completed ...
2018-04-24 11:57:39 PM Replacing ./OA-G3-0/AMBER/APR/windows/a014 with ./OA-G3-0/AMBER/APR/windows/p000 in ['./OA-G3-0/AMBER/APR/windows/a000', './OA-G3-0/AMBER/APR/windows/a001', './OA-G3-0/AMBER/APR/windows/a002', './OA-G3-0/AMBER/APR/windows/a003', './OA-G3-0/AMBER/APR/windows/a004', './OA-G3-0/AMBER/APR/windows/a005', './OA-G3-0/AMBER/APR/windows/a006', './OA-G3-0/AMBER/APR/windows/a007', './OA-G3-0/AMBER/APR/windows/a008', './OA-G3-0/AMBER/APR/windows/a009', './OA-G3-0/AMBER/APR/windows/a010', './OA-G3-0/AMBER/APR/windows/a011', './OA-G3-0/AMBER/APR/windows/a012', './OA-G3-0/AMBER/APR/windows/a01

2018-04-24 11:57:40 PM Reading restraint data for ./OA-G3-0/AMBER/APR/windows/p038...
2018-04-24 11:57:40 PM Reading restraint data for ./OA-G3-0/AMBER/APR/windows/p039...
2018-04-24 11:57:40 PM Reading restraint data for ./OA-G3-0/AMBER/APR/windows/p040...
2018-04-24 11:57:40 PM Reading restraint data for ./OA-G3-0/AMBER/APR/windows/p041...
2018-04-24 11:57:40 PM Reading restraint data for ./OA-G3-0/AMBER/APR/windows/p042...
2018-04-24 11:57:40 PM Reading restraint data for ./OA-G3-0/AMBER/APR/windows/p043...
2018-04-24 11:57:40 PM Reading restraint data for ./OA-G3-0/AMBER/APR/windows/p044...
2018-04-24 11:57:40 PM Reading restraint data for ./OA-G3-0/AMBER/APR/windows/p045...


TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [115]:
def clean(path):
    minimization_files = glob.glob(os.path.join(path, 'minimize*'))
    equilibration_files = glob.glob(os.path.join(path, 'equil*'))
    production_files = glob.glob(os.path.join(path, 'prod*'))
    
    files = minimization_files + equilibration_files + production_files
    print(files)
    
    for file in files:
        if os.path.isfile(file):
            print(f'Deleting {file}')
            os.remove(file)

In [116]:
for complx in g3_complexes:
    for window in window_list:
        # clean(os.path.join(complx, 'AMBER', 'APR', 'windows', window))

['./OA-G3-0/AMBER/APR/windows/a000/minimize.in', './OA-G3-0/AMBER/APR/windows/a000/minimize.out', './OA-G3-0/AMBER/APR/windows/a000/minimize.rst7', './OA-G3-0/AMBER/APR/windows/a000/minimize.mdinfo', './OA-G3-0/AMBER/APR/windows/a000/equil.0.in', './OA-G3-0/AMBER/APR/windows/a000/equil.0.out', './OA-G3-0/AMBER/APR/windows/a000/equil.0.nc', './OA-G3-0/AMBER/APR/windows/a000/equil.0.mden', './OA-G3-0/AMBER/APR/windows/a000/equil.0.rst7', './OA-G3-0/AMBER/APR/windows/a000/equil.0.mdinfo', './OA-G3-0/AMBER/APR/windows/a000/equil.rst7', './OA-G3-0/AMBER/APR/windows/a000/prod.001.in', './OA-G3-0/AMBER/APR/windows/a000/prod.001.out', './OA-G3-0/AMBER/APR/windows/a000/prod.001.nc', './OA-G3-0/AMBER/APR/windows/a000/prod.001.mden', './OA-G3-0/AMBER/APR/windows/a000/prod.001.mdinfo', './OA-G3-0/AMBER/APR/windows/a000/prod.001.rst7']
Deleting ./OA-G3-0/AMBER/APR/windows/a000/minimize.in
Deleting ./OA-G3-0/AMBER/APR/windows/a000/minimize.out
Deleting ./OA-G3-0/AMBER/APR/windows/a000/minimize.rst7


Deleting ./OA-G3-0/AMBER/APR/windows/p018/prod.001.out
Deleting ./OA-G3-0/AMBER/APR/windows/p018/prod.001.nc
Deleting ./OA-G3-0/AMBER/APR/windows/p018/prod.001.mden
Deleting ./OA-G3-0/AMBER/APR/windows/p018/prod.001.mdinfo
Deleting ./OA-G3-0/AMBER/APR/windows/p018/prod.001.rst7
['./OA-G3-0/AMBER/APR/windows/p019/minimize.in', './OA-G3-0/AMBER/APR/windows/p019/minimize.out', './OA-G3-0/AMBER/APR/windows/p019/minimize.rst7', './OA-G3-0/AMBER/APR/windows/p019/minimize.mdinfo', './OA-G3-0/AMBER/APR/windows/p019/equil.0.in', './OA-G3-0/AMBER/APR/windows/p019/equil.0.out', './OA-G3-0/AMBER/APR/windows/p019/equil.0.nc', './OA-G3-0/AMBER/APR/windows/p019/equil.0.mden', './OA-G3-0/AMBER/APR/windows/p019/equil.0.rst7', './OA-G3-0/AMBER/APR/windows/p019/equil.0.mdinfo', './OA-G3-0/AMBER/APR/windows/p019/equil.rst7', './OA-G3-0/AMBER/APR/windows/p019/prod.001.in', './OA-G3-0/AMBER/APR/windows/p019/prod.001.out', './OA-G3-0/AMBER/APR/windows/p019/prod.001.nc', './OA-G3-0/AMBER/APR/windows/p019/prod

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [124]:
pt.load('./OA-G3-0/AMBER/APR/windows/a000/prod.001.nc', './OA-G3-0/AMBER/APR/windows/a000/vac.prmtop')

pytraj.Trajectory, 2000 frames: 
Size: 0.008985 (GB)
<Topology: 201 atoms, 2 residues, 2 mols, PBC with box type = ortho>
           

In [119]:
stripped.topology

<Topology: 204 atoms, 5 residues, 5 mols, PBC with box type = ortho>

## Run so that there is at least `prod.002.nc` in each folder!

In [132]:
for complx in g3_complexes:
    
    traj = pt.load(os.path.join(complx, 'AMBER', 'solvate.rst7'), 
                    os.path.join(complx, 'AMBER', 'solvate.prmtop'))

    pull_initial = pt.distance(traj, ' '.join([dummy_anchors[0], guest_anchors[0]]))[0]

    print(f'Set pull offset ({pull_initial:1.2f} A)')

    pull_distances = [float(i) + pull_initial for i in pull_string.split()]
    
    guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                        ]

    guest_restraint_targets = [pull_initial,
                               180.0,
                               180.0
                              ]
    guest_restraint_target_final = [pull_distances[-1], 180.0, 180.0]
    guest_restraint_distance_fc = 5.0 # kcal/mol-A**2
    guest_restraint_angle_fc = 100.0  # kcal/mol-rad**2

    hg = pmd.load_file(os.path.join(complx, 'AMBER', 'solvate.prmtop'),
                   os.path.join(complx, 'AMBER', 'solvate.rst7'),
                   structure=True)

    guest_restraints = []
    for index, atoms in enumerate(guest_restraint_atoms): 
        if len(atoms) > 2:
            angle = True
        else:
            angle = False
        this = DAT_restraint()
        this.auto_apr = True
        this.amber_index = True
        this.topology = hg
        this.mask1 = atoms[0]
        this.mask2 = atoms[1]
        if angle:
            this.mask3 = atoms[2]
            this.attach['fc_final'] = guest_restraint_angle_fc
        else:
            this.attach['fc_final'] = guest_restraint_distance_fc
        this.attach['target'] = guest_restraint_targets[index]
        this.attach['fraction_list'] = attach_fractions

        this.pull['target_final'] = guest_restraint_target_final[index]
        this.pull['num_windows'] = windows[1]
        this.initialize()

        guest_restraints.append(this)
        
    structure = pt.load(os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'solvate.rst7'), 
                       os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'solvate.prmtop'))

    stripped = structure.strip(':WAT,:Na+,:Cl-')
    
    print(f'Simulating {complx}')

    for i_win, win in enumerate(window_list):
        phase = phase_dict[win[0]]
        win_path = os.path.join(complx, 'AMBER', 'APR', 'windows', win)

        if not os.path.isfile(os.path.join(win_path, 'prod.002.nc')):

            sim = Simulation()
            sim.executable = 'pmemd.cuda'
            sim.CUDA_VISIBLE_DEVICES = 1
            sim.path = win_path
            sim.topology = 'solvate.prmtop'
            sim.restraint_file = 'disang.rest'

            sim.config_pbc_md()

            sim.prefix = 'prod.002'

            sim.inpcrd = 'prod.001.rst7'
            sim.ref = 'solvate.rst7'
            sim.cntrl['ntx'] = 5
            sim.cntrl['irest'] = 1
            sim.cntrl['nstlim'] = itr_nstlim
            sim.cntrl['ntwr'] = itr_nstlim
            sim.cntrl['ntwx'] = 250
            sim.cntrl['ntwprt'] = ntwprt
            sim.cntrl['ntxo'] = 2
            sim.restraint_file = 'disang.rest'
            sim.run()

        else:
            print(f'Window {win} has `prod.002.nc`...')

Set pull offset (8.03 A)


2018-04-26 05:12:37 PM Calculating attach targets and force constants...
2018-04-26 05:12:37 PM Attach, Method #3
2018-04-26 05:12:37 PM Calculating pull targets and force constants...
2018-04-26 05:12:37 PM Pull, Method #1
2018-04-26 05:12:37 PM Calculating release targets and force constants...
2018-04-26 05:12:37 PM No restraint info set for the release phase! Skipping...
2018-04-26 05:12:37 PM Number of attach windows = 15
2018-04-26 05:12:37 PM Number of pull windows = 46
2018-04-26 05:12:37 PM This restraint will be skipped in the release phase
2018-04-26 05:12:37 PM Assigning atom indices...
2018-04-26 05:12:37 PM There are 1 atoms in the mask :3  ...
2018-04-26 05:12:37 PM There are 1 atoms in the mask :2@C1  ...
2018-04-26 05:12:37 PM Calculating attach targets and force constants...
2018-04-26 05:12:37 PM Attach, Method #3
2018-04-26 05:12:37 PM Calculating pull targets and force constants...
2018-04-26 05:12:37 PM Pull, Method #1
2018-04-26 05:12:37 PM Calculating release ta

Simulating ./OA-G3-0
Window a000 has `prod.002.nc`...
Window a001 has `prod.002.nc`...


2018-04-26 05:15:18 PM MD completed ...
2018-04-26 05:15:18 PM Writing prod.002.in
2018-04-26 05:15:18 PM Running MD at ./OA-G3-0/AMBER/APR/windows/a003
2018-04-26 05:15:18 PM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r prod.002.rst7 -x prod.002.nc -inf prod.002.mdinfo -e prod.002.mden
2018-04-26 05:17:56 PM MD completed ...
2018-04-26 05:17:56 PM Writing prod.002.in
2018-04-26 05:17:56 PM Running MD at ./OA-G3-0/AMBER/APR/windows/a004
2018-04-26 05:17:56 PM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r prod.002.rst7 -x prod.002.nc -inf prod.002.mdinfo -e prod.002.mden
2018-04-26 05:20:31 PM MD completed ...
2018-04-26 05:20:31 PM Writing prod.002.in
2018-04-26 05:20:31 PM Running MD at ./OA-G3-0/AMBER/APR/windows/a005
2018-04-26 05:20:31 PM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r pr

Window a008 has `prod.002.nc`...
Window a009 has `prod.002.nc`...
Window a010 has `prod.002.nc`...
Window a011 has `prod.002.nc`...
Window a012 has `prod.002.nc`...
Window a013 has `prod.002.nc`...
Window p000 has `prod.002.nc`...
Window p001 has `prod.002.nc`...
Window p002 has `prod.002.nc`...
Window p003 has `prod.002.nc`...


...

2018-04-27 01:26:17 AM Calculating attach targets and force constants...
2018-04-27 01:26:17 AM Attach, Method #3
2018-04-27 01:26:17 AM Calculating pull targets and force constants...
2018-04-27 01:26:17 AM Pull, Method #1
2018-04-27 01:26:17 AM Calculating release targets and force constants...
2018-04-27 01:26:17 AM No restraint info set for the release phase! Skipping...
2018-04-27 01:26:17 AM Number of attach windows = 15
2018-04-27 01:26:17 AM Number of pull windows = 46
2018-04-27 01:26:17 AM This restraint will be skipped in the release phase
2018-04-27 01:26:17 AM Assigning atom indices...
2018-04-27 01:26:17 AM There are 1 atoms in the mask :3  ...
2018-04-27 01:26:17 AM There are 1 atoms in the mask :2@C1  ...
2018-04-27 01:26:17 AM Calculating attach targets and force constants...
2018-04-27 01:26:17 AM Attach, Method #3
2018-04-27 01:26:17 AM Calculating pull targets and force constants...
2018-04-27 01:26:17 AM Pull, Method #1
2018-04-27 01:26:17 AM Calculating release ta

Simulating ./OA-G3-4


2018-04-27 01:28:56 AM MD completed ...
2018-04-27 01:28:56 AM Writing prod.002.in
2018-04-27 01:28:56 AM Running MD at ./OA-G3-4/AMBER/APR/windows/a001
2018-04-27 01:28:56 AM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r prod.002.rst7 -x prod.002.nc -inf prod.002.mdinfo -e prod.002.mden
2018-04-27 01:31:35 AM MD completed ...
2018-04-27 01:31:35 AM Writing prod.002.in
2018-04-27 01:31:35 AM Running MD at ./OA-G3-4/AMBER/APR/windows/a002
2018-04-27 01:31:35 AM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r prod.002.rst7 -x prod.002.nc -inf prod.002.mdinfo -e prod.002.mden
2018-04-27 01:34:15 AM MD completed ...
2018-04-27 01:34:15 AM Writing prod.002.in
2018-04-27 01:34:15 AM Running MD at ./OA-G3-4/AMBER/APR/windows/a003
2018-04-27 01:34:15 AM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r pr

2018-04-27 02:26:55 AM MD completed ...
2018-04-27 02:26:55 AM Writing prod.002.in
2018-04-27 02:26:55 AM Running MD at ./OA-G3-4/AMBER/APR/windows/p010
2018-04-27 02:26:55 AM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r prod.002.rst7 -x prod.002.nc -inf prod.002.mdinfo -e prod.002.mden
2018-04-27 02:29:33 AM MD completed ...
2018-04-27 02:29:33 AM Writing prod.002.in
2018-04-27 02:29:33 AM Running MD at ./OA-G3-4/AMBER/APR/windows/p011
2018-04-27 02:29:33 AM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r prod.002.rst7 -x prod.002.nc -inf prod.002.mdinfo -e prod.002.mden
2018-04-27 02:32:10 AM MD completed ...
2018-04-27 02:32:10 AM Writing prod.002.in
2018-04-27 02:32:10 AM Running MD at ./OA-G3-4/AMBER/APR/windows/p012
2018-04-27 02:32:10 AM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r pr

2018-04-27 03:27:02 AM Writing prod.002.in
2018-04-27 03:27:02 AM Running MD at ./OA-G3-4/AMBER/APR/windows/p033
2018-04-27 03:27:02 AM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r prod.002.rst7 -x prod.002.nc -inf prod.002.mdinfo -e prod.002.mden
2018-04-27 03:29:37 AM MD completed ...
2018-04-27 03:29:37 AM Writing prod.002.in
2018-04-27 03:29:37 AM Running MD at ./OA-G3-4/AMBER/APR/windows/p034
2018-04-27 03:29:37 AM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r prod.002.rst7 -x prod.002.nc -inf prod.002.mdinfo -e prod.002.mden
2018-04-27 03:32:13 AM MD completed ...
2018-04-27 03:32:13 AM Writing prod.002.in
2018-04-27 03:32:13 AM Running MD at ./OA-G3-4/AMBER/APR/windows/p035
2018-04-27 03:32:13 AM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r prod.002.rst7 -x prod.002.nc -inf prod.002

## Re-run a few windows

In [136]:
stats = os.stat(os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'prod.002.nc'))

In [145]:
for complx in g3_complexes:
    
    traj = pt.load(os.path.join(complx, 'AMBER', 'solvate.rst7'), 
                    os.path.join(complx, 'AMBER', 'solvate.prmtop'))

    pull_initial = pt.distance(traj, ' '.join([dummy_anchors[0], guest_anchors[0]]))[0]

    print(f'Set pull offset ({pull_initial:1.2f} A)')

    pull_distances = [float(i) + pull_initial for i in pull_string.split()]
    
    guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                        ]

    guest_restraint_targets = [pull_initial,
                               180.0,
                               180.0
                              ]
    guest_restraint_target_final = [pull_distances[-1], 180.0, 180.0]
    guest_restraint_distance_fc = 5.0 # kcal/mol-A**2
    guest_restraint_angle_fc = 100.0  # kcal/mol-rad**2

    hg = pmd.load_file(os.path.join(complx, 'AMBER', 'solvate.prmtop'),
                   os.path.join(complx, 'AMBER', 'solvate.rst7'),
                   structure=True)

    guest_restraints = []
    for index, atoms in enumerate(guest_restraint_atoms): 
        if len(atoms) > 2:
            angle = True
        else:
            angle = False
        this = DAT_restraint()
        this.auto_apr = True
        this.amber_index = True
        this.topology = hg
        this.mask1 = atoms[0]
        this.mask2 = atoms[1]
        if angle:
            this.mask3 = atoms[2]
            this.attach['fc_final'] = guest_restraint_angle_fc
        else:
            this.attach['fc_final'] = guest_restraint_distance_fc
        this.attach['target'] = guest_restraint_targets[index]
        this.attach['fraction_list'] = attach_fractions

        this.pull['target_final'] = guest_restraint_target_final[index]
        this.pull['num_windows'] = windows[1]
        this.initialize()

        guest_restraints.append(this)
        
    structure = pt.load(os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'solvate.rst7'), 
                       os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'solvate.prmtop'))

    stripped = structure.strip(':WAT,:Na+,:Cl-')
    
    print(f'Simulating {complx}')

    for i_win, win in enumerate(window_list):
        phase = phase_dict[win[0]]
        win_path = os.path.join(complx, 'AMBER', 'APR', 'windows', win)

        stats = os.stat(os.path.join(win_path, 'prod.002.nc'))
        size = stats.st_size
        
        if size < 5000808:
            #print('Delete {}'.format(os.path.join(win_path, 'prod.002.nc')))
            os.remove(os.path.join(win_path, 'prod.002.out'))
            os.remove(os.path.join(win_path, 'prod.002.in'))
        
        if not os.path.isfile(os.path.join(win_path, 'prod.002.nc')) or size < 5000808:
            print('Re-running {}...'.format(os.path.join(win_path, 'prod.002.nc')))

            sim = Simulation()
            sim.executable = 'pmemd.cuda'
            sim.CUDA_VISIBLE_DEVICES = 1
            sim.path = win_path
            sim.topology = 'solvate.prmtop'
            sim.restraint_file = 'disang.rest'

            sim.config_pbc_md()

            sim.prefix = 'prod.002'

            sim.inpcrd = 'prod.001.rst7'
            sim.ref = 'solvate.rst7'
            sim.cntrl['ntx'] = 5
            sim.cntrl['irest'] = 1
            sim.cntrl['nstlim'] = itr_nstlim
            sim.cntrl['ntwr'] = itr_nstlim
            sim.cntrl['ntwx'] = 250
            sim.cntrl['ntwprt'] = ntwprt
            sim.cntrl['ntxo'] = 2
            sim.restraint_file = 'disang.rest'
            sim.run()

        else:
            print(f'Window {win} has `prod.002.nc`...')

Set pull offset (8.03 A)


2018-04-27 01:24:04 PM Calculating attach targets and force constants...
2018-04-27 01:24:04 PM Attach, Method #3
2018-04-27 01:24:04 PM Calculating pull targets and force constants...
2018-04-27 01:24:04 PM Pull, Method #1
2018-04-27 01:24:04 PM Calculating release targets and force constants...
2018-04-27 01:24:04 PM No restraint info set for the release phase! Skipping...
2018-04-27 01:24:04 PM Number of attach windows = 15
2018-04-27 01:24:04 PM Number of pull windows = 46
2018-04-27 01:24:04 PM This restraint will be skipped in the release phase
2018-04-27 01:24:04 PM Assigning atom indices...
2018-04-27 01:24:04 PM There are 1 atoms in the mask :3  ...
2018-04-27 01:24:04 PM There are 1 atoms in the mask :2@C1  ...
2018-04-27 01:24:04 PM Calculating attach targets and force constants...
2018-04-27 01:24:04 PM Attach, Method #3
2018-04-27 01:24:04 PM Calculating pull targets and force constants...
2018-04-27 01:24:04 PM Pull, Method #1
2018-04-27 01:24:04 PM Calculating release ta

Simulating ./OA-G3-0
Window a000 has `prod.002.nc`...
Window a001 has `prod.002.nc`...
Re-running ./OA-G3-0/AMBER/APR/windows/a002/prod.002.nc...


2018-04-27 01:26:40 PM MD completed ...
2018-04-27 01:26:40 PM Writing prod.002.in
2018-04-27 01:26:40 PM Running MD at ./OA-G3-0/AMBER/APR/windows/a003
2018-04-27 01:26:40 PM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r prod.002.rst7 -x prod.002.nc -inf prod.002.mdinfo -e prod.002.mden


Re-running ./OA-G3-0/AMBER/APR/windows/a003/prod.002.nc...


...

2018-04-28 12:15:47 AM MD completed ...
2018-04-28 12:15:47 AM Writing prod.002.in
2018-04-28 12:15:47 AM Running MD at ./OA-G3-4/AMBER/APR/windows/p044
2018-04-28 12:15:47 AM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r prod.002.rst7 -x prod.002.nc -inf prod.002.mdinfo -e prod.002.mden


Re-running ./OA-G3-4/AMBER/APR/windows/p044/prod.002.nc...


2018-04-28 12:18:27 AM MD completed ...
2018-04-28 12:18:27 AM Writing prod.002.in
2018-04-28 12:18:27 AM Running MD at ./OA-G3-4/AMBER/APR/windows/p045
2018-04-28 12:18:27 AM Exec line: pmemd.cuda -O -p solvate.prmtop -ref solvate.rst7 -c prod.001.rst7 -i prod.002.in -o prod.002.out -r prod.002.rst7 -x prod.002.nc -inf prod.002.mdinfo -e prod.002.mden


Re-running ./OA-G3-4/AMBER/APR/windows/p045/prod.002.nc...


2018-04-28 12:21:05 AM MD completed ...


In [8]:
for complx in g3_complexes:
    
    traj = pt.load(os.path.join(complx, 'AMBER', 'solvate.rst7'), 
                    os.path.join(complx, 'AMBER', 'solvate.prmtop'))

    pull_initial = pt.distance(traj, ' '.join([dummy_anchors[0], guest_anchors[0]]))[0]

    print(f'Set pull offset ({pull_initial:1.2f} A)')

    pull_distances = [float(i) + pull_initial for i in pull_string.split()]
    
    guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                        ]

    guest_restraint_targets = [pull_initial,
                               180.0,
                               180.0
                              ]
    guest_restraint_target_final = [pull_distances[-1], 180.0, 180.0]
    guest_restraint_distance_fc = 5.0 # kcal/mol-A**2
    guest_restraint_angle_fc = 100.0  # kcal/mol-rad**2

    hg = pmd.load_file(os.path.join(complx, 'AMBER', 'solvate.prmtop'),
                   os.path.join(complx, 'AMBER', 'solvate.rst7'),
                   structure=True)

    guest_restraints = []
    for index, atoms in enumerate(guest_restraint_atoms): 
        if len(atoms) > 2:
            angle = True
        else:
            angle = False
        this = DAT_restraint()
        this.auto_apr = True
        this.amber_index = True
        this.topology = hg
        this.mask1 = atoms[0]
        this.mask2 = atoms[1]
        if angle:
            this.mask3 = atoms[2]
            this.attach['fc_final'] = guest_restraint_angle_fc
        else:
            this.attach['fc_final'] = guest_restraint_distance_fc
        this.attach['target'] = guest_restraint_targets[index]
        this.attach['fraction_list'] = attach_fractions

        this.pull['target_final'] = guest_restraint_target_final[index]
        this.pull['num_windows'] = windows[1]
        this.initialize()

        guest_restraints.append(this)
        
    structure = pt.load(os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'solvate.rst7'), 
                       os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'solvate.prmtop'))

    stripped = structure.strip(':WAT,:Na+,:Cl-')
    
    print(f'Simulating {complx}')

    for i_win, win in enumerate(window_list):
        phase = phase_dict[win[0]]
        win_path = os.path.join(complx, 'AMBER', 'APR', 'windows', win)

        # Loop through prod.003.nc through prod.005.nc,
        # if they exist, delete them (they may have used the wrong restart),
        # and then simulate.
        iterations = np.arange(3, 6)
        for iteration in iterations:
            
            if os.path.exists(os.path.join(win_path, 'prod.{:03d}.nc'.format(iteration))):
                print('Deleting {}'.format(os.path.join(win_path, 'prod.{:03d}.nc'.format(iteration))))
                try:
                    os.remove(os.path.join(win_path, 'prod.{:03d}.out'.format(iteration)))
                except:
                    print('prod.{:03d}.out already removed...'.format(iteration))
                try:
                    os.remove(os.path.join(win_path, 'prod.{:03d}.in'.format(iteration)))
                except:
                    print('prod.{:03d}.in already removed...'.format(iteration))

            else:
                pass
                    
            sim = Simulation()
            sim.executable = 'pmemd.cuda'
            sim.CUDA_VISIBLE_DEVICES = 1
            sim.path = win_path
            sim.topology = 'solvate.prmtop'
            sim.restraint_file = 'disang.rest'
            sim.config_pbc_md()

            sim.prefix = 'prod.{:03d}'.format(iteration)
            sim.inpcrd = 'prod.{:03d}.rst7'.format(iteration - 1)

            sim.ref = 'solvate.rst7'
            sim.cntrl['ntx'] = 5
            sim.cntrl['irest'] = 1
            sim.cntrl['nstlim'] = itr_nstlim
            sim.cntrl['ntwr'] = itr_nstlim
            sim.cntrl['ntwx'] = 250
            sim.cntrl['ntwprt'] = ntwprt
            sim.cntrl['ntxo'] = 2
            sim.restraint_file = 'disang.rest'
            sim.run()


Set pull offset (8.03 A)
Simulating ./OA-G3-0
Set pull offset (5.63 A)
Simulating ./OA-G3-1
Set pull offset (7.28 A)
Simulating ./OA-G3-2
Set pull offset (7.46 A)
Simulating ./OA-G3-3
Set pull offset (6.83 A)
Simulating ./OA-G3-4


RuntimeError: n_atoms = 0: make sure to load correct Topology filename or load supported topology (pdb, amber parm, psf, ...)

## Run `a000` a bit longer to help convergence among the complexes...

In [10]:
for complx in g3_complexes:
    
    traj = pt.load(os.path.join(complx, 'AMBER', 'solvate.rst7'), 
                    os.path.join(complx, 'AMBER', 'solvate.prmtop'))

    pull_initial = pt.distance(traj, ' '.join([dummy_anchors[0], guest_anchors[0]]))[0]

    print(f'Set pull offset ({pull_initial:1.2f} A)')

    pull_distances = [float(i) + pull_initial for i in pull_string.split()]
    
    guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                        ]

    guest_restraint_targets = [pull_initial,
                               180.0,
                               180.0
                              ]
    guest_restraint_target_final = [pull_distances[-1], 180.0, 180.0]
    guest_restraint_distance_fc = 5.0 # kcal/mol-A**2
    guest_restraint_angle_fc = 100.0  # kcal/mol-rad**2

    hg = pmd.load_file(os.path.join(complx, 'AMBER', 'solvate.prmtop'),
                   os.path.join(complx, 'AMBER', 'solvate.rst7'),
                   structure=True)

    guest_restraints = []
    for index, atoms in enumerate(guest_restraint_atoms): 
        if len(atoms) > 2:
            angle = True
        else:
            angle = False
        this = DAT_restraint()
        this.auto_apr = True
        this.amber_index = True
        this.topology = hg
        this.mask1 = atoms[0]
        this.mask2 = atoms[1]
        if angle:
            this.mask3 = atoms[2]
            this.attach['fc_final'] = guest_restraint_angle_fc
        else:
            this.attach['fc_final'] = guest_restraint_distance_fc
        this.attach['target'] = guest_restraint_targets[index]
        this.attach['fraction_list'] = attach_fractions

        this.pull['target_final'] = guest_restraint_target_final[index]
        this.pull['num_windows'] = windows[1]
        this.initialize()

        guest_restraints.append(this)
        
    structure = pt.load(os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'solvate.rst7'), 
                       os.path.join(complx, 'AMBER', 'APR', 'windows', 'a000', 'solvate.prmtop'))

    stripped = structure.strip(':WAT,:Na+,:Cl-')
    
    print(f'Simulating {complx}')

    win = 'a000'
    phase = phase_dict[win[0]]
    win_path = os.path.join(complx, 'AMBER', 'APR', 'windows', win)

    iterations = np.arange(6, 10)
    for iteration in iterations:
        print(f'Iteration {iteration}...')

        sim = Simulation()
        sim.executable = 'pmemd.cuda'
        sim.CUDA_VISIBLE_DEVICES = 1
        sim.path = win_path
        sim.topology = 'solvate.prmtop'
        sim.restraint_file = 'disang.rest'
        sim.config_pbc_md()

        sim.prefix = 'prod.{:03d}'.format(iteration)
        sim.inpcrd = 'prod.{:03d}.rst7'.format(iteration - 1)

        sim.ref = 'solvate.rst7'
        sim.cntrl['ntx'] = 5
        sim.cntrl['irest'] = 1
        sim.cntrl['nstlim'] = itr_nstlim
        sim.cntrl['ntwr'] = itr_nstlim
        sim.cntrl['ntwx'] = 250
        sim.cntrl['ntwprt'] = ntwprt
        sim.cntrl['ntxo'] = 2
        sim.restraint_file = 'disang.rest'
        sim.run()


Set pull offset (8.03 A)
Simulating ./OA-G3-0
Iteration 6...
Iteration 7...
Iteration 8...
Iteration 9...
Set pull offset (5.63 A)
Simulating ./OA-G3-1
Iteration 6...
Iteration 7...
Iteration 8...
Iteration 9...
Set pull offset (7.28 A)
Simulating ./OA-G3-2
Iteration 6...
Iteration 7...
Iteration 8...
Iteration 9...
Set pull offset (7.46 A)
Simulating ./OA-G3-3
Iteration 6...
Iteration 7...
Iteration 8...
Iteration 9...
Set pull offset (6.83 A)
Simulating ./OA-G3-4
Iteration 6...
Iteration 7...
Iteration 8...
Iteration 9...


RuntimeError: n_atoms = 0: make sure to load correct Topology filename or load supported topology (pdb, amber parm, psf, ...)